In [ ]:
from flask import Flask, request, jsonify
import librosa
import numpy as np
from joblib import load

app = Flask(__name__)

# Load the trained Random Forest model
random_forest_model = load('random_forest_model.joblib')

# Function to extract features from audio
def get_feature_vector(audio_file):
    y, sr = librosa.load(audio_file, sr=None)  # Load audio file
    
    # Extract features
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
    lpc = np.std(librosa.lpc(y, order=16))  # Specify the order parameter
    
    # Additional features
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=y, sr=sr))
    tempo = librosa.beat.tempo(y=y, sr=sr)[0]
    
    # Combine features into a single feature vector
    feature_vector = [spectral_centroid, spectral_bandwidth, spectral_rolloff,
                      zero_crossing_rate, *mfccs, lpc, spectral_contrast,
                      chroma_stft, tempo]
    return feature_vector

@app.route('/extract-features', methods=['POST'])
def extract_features():
    # Receive the recorded audio file from the UI
    audio_file = request.files['audio']
    audio_file.save('recorded_audio.wav')
    
    # Extract features from the recorded audio file
    feature_vector = get_feature_vector('recorded_audio.wav')
    
    # Use the trained model to classify if it is a concussion or not
    prediction = random_forest_model.predict([feature_vector])[0]
    
    # Return the prediction as JSON
    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(debug=True)
